## Dual-arm cluster tool with MLIF flow wafers

* Process times sampled between 10-300 seconds
* Baseline robot movement sequence uses Swap sequence
* Trained model checkpoint file loaded from ./saved_models/checkpoint_s4sp6.pt

In [1]:
import re
import os
import sys
import copy
import time
import random
import itertools
import numpy as np
import pandas as pd
import torch
import argparse
from envs.clustertool import NoncyclicClusterToolEnv as Env
from envs.algorithms.swap import get_policy
from model.CONCAT.model_concat import CONCATNet as CONCATModel


import warnings
warnings.filterwarnings("ignore", category=UserWarning)

STAGE_LIST = {
    's3s3': [1, 1, 1],
    's4s4': [1, 1, 1, 1],
    's2p5': [2, 3],
    's3sp4': [1, 2, 1],
    's3sp6': [1, 3, 2],
    's4sp6': [1, 2, 2, 1],
}


def settings(args):
    # seed fix
    DEBUG_MODE = True
    USE_CUDA = False
    CUDA_DEVICE_NUM = 0
    SEED = 1000
    torch.backends.cudnn.deterministic = True
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

    # env param setting
    env_params = {
            'arm_type': args.arm_type,
            'stage': STAGE_LIST[args.stage_type],
            'init_partial_loading': [0 for _ in range(len(STAGE_LIST[args.stage_type]))],
            'stage_z': args.z,
            'strategy': args.a,
            'min_process_time': args.min_process_time,
            'max_process_time': args.max_process_time,
            'min_purge_time': args.min_purge_time ,
            'max_purge_time': args.max_purge_time,
            'purge_type': args.purge_type,
            'loadport_capacity': 2,
            'num_foup': args.num_foup,
            'foup_size': args.foup_size,
            'num_lot_type': 5,
            'lot_dist': args.foup_type,
            'lot_variance': False,
            'consider_lot_type': args.consider_lot_type,
            }

    # model param setting
    model_params = {
        "purge": True if env_params['max_purge_time'] != 0 else False,
        'input_action': 'wafer',
        'embedding_dim': 256,
        'sqrt_embedding_dim': 256**(1/2),
        'encoder_layer_num': 3,
        'qkv_dim': 16,
        'sqrt_qkv_dim': 16**(1/2),
        'head_num': 16,
        'logit_clipping': 10,
        'ff_hidden_dim': 512,
        'ms_hidden_dim': 16,
        'ms_layer1_init': (1/2)**(1/2),
        'ms_layer2_init': (1/16)**(1/2),
        'eval_type': 'argmax',
        'normalize': 'instance' if env_params['num_lot_type'] > 1 else 'batch',
    }

    # test param setting
    const_type = 'purge' if model_params['purge'] else 'basic'
    test_params = {
        'use_cuda': USE_CUDA,
        'cuda_device_num': CUDA_DEVICE_NUM,
        'model_load': {
            'enable': args.use_trained_model,
            'use_latest_model': args.use_latest_model,
            'epoch': args.epoch
        },
        'multi_run_size': 1,
        'problem_count': args.num_test_problem,
        'test_batch_size': args.num_test_problem
    }

    # CUDA setting
    USE_CUDA = test_params['use_cuda']
    if USE_CUDA:
        cuda_device_num = test_params['cuda_device_num']
        torch.cuda.set_device(cuda_device_num)
        device = torch.device('cuda', cuda_device_num)
        #torch.set_default_tensor_type('torch.cuda.FloatTensor')
    else:
        device = torch.device('cpu')
    model_params['device'] = device

    # call env and model
    env = Env(**env_params)
    drl_model = CONCATModel(env, **env_params, **model_params)
    drl_model.params['eval_type'] = 'argmax'
    drl_model.eval()
    test_params['model_load']['enable'] = True 
    if test_params['model_load']['enable']:
        #current_dir = os.path.dirname(os.path.abspath(__file__))
        #checkpoint_fullname = f'{current_dir}/saved_models/checkpoint_s4sp6.pt'
        checkpoint_fullname = f'./saved_models/checkpoint_s4sp6.pt'
        checkpoint = torch.load(checkpoint_fullname, map_location=device)
        drl_model.load_state_dict(checkpoint['model_state_dict'])
        print(f'>>> trained Model {checkpoint_fullname} Loaded....')

    
    env = Env(**env_params)
    state = env.reset(test_params['test_batch_size'], device=device)

    # baseline robot move sequence rule settings
    if env.arm_type == 'single':
        if env.purge_constraint:
            baseline_policy = 'backward_z'
        else:
            baseline_policy = 'backward'

    else:
        if env.purge_constraint:
            baseline_policy = 'swap_a_z'
        else:
            baseline_policy = 'swap'
            
    print(f'>>> use conventional robot sequence = {baseline_policy}')
    env.lot_release_rule = args.input_sequencing_rule
    base_model = get_policy(baseline_policy, env)

    return env, state, base_model, drl_model, env_params, model_params, test_params


def compare_performance(args):
    env, state, rule, model, ep, mp, tp = settings(args)

    rolling_foup_cnt = args.rolling_foup_cnt

    # run baseline
    ###################################################
    benv = copy.deepcopy(env)
    bstate = copy.deepcopy(state)
    step_cnt = 0 
    
    done = False
    while not done:
        action = rule(benv, bstate)
        bstate = benv.step(action, rule=True, show=args.show)

        #foup_switch = benv.wafer.loc[:, :rolling_foup_cnt, :].sum(dim=-1).sum(dim=-1) == - rolling_foup_cnt*benv.foup_size
        done = benv.done.all()
        step_cnt += 1

    
    # run DRL
    ###################################################
    renv = copy.deepcopy(env)
    rstate = copy.deepcopy(state)
    model.encoding(renv, rstate)

    # rollout
    while not rstate.done.all():
        action, _ = model(renv, rstate)
        rstate = renv.step(action, show=args.show)

    # calculate throughput 
    ###################################################
    base_makespan = args.min_process_time + (args.max_process_time - args.min_process_time) * (benv.clock)
    drl_makespan = args.min_process_time + (args.max_process_time - args.min_process_time) * (renv.clock)
    
    return base_makespan, drl_makespan, env

def main():
    parser = argparse.ArgumentParser(description='Train a model with a specific number of lot types')
    # ENV SETTINGS
    parser.add_argument('--arm_type', type=str, default='dual', help='arm type')
    parser.add_argument('--stage_type', type=str, default='s4sp6', help='stage type s3sp4, s3sp6, s4sp6')
    parser.add_argument('--min_process_time', type=int, default=10, help='process time')
    parser.add_argument('--max_process_time', type=int, default=300, help='process time')
    parser.add_argument('--min_purge_time', type=int, default=0, help='purge time')
    parser.add_argument('--max_purge_time', type=int, default=0, help='purge time')
    parser.add_argument('--num_foup', type=int, default=5, help='num_foup')
    parser.add_argument('--foup_size', type=int, default=25, help='foup size')
    parser.add_argument('--foup_type', type=str, default='multi_lot_imbalanced', help='multi_lot_imbalanced, multi_lot_balanced')
    parser.add_argument('--consider_lot_type', type=int, default=5, help='foup type')
    parser.add_argument('--purge_type', type=str, default='long', help='short, long')

    # MODEL SETTINGS
    parser.add_argument('--use_trained_model', type=bool, default=True, help='use trained model')
    parser.add_argument('--use_latest_model', type=bool, default=True, help='use latest trained model')
    parser.add_argument('--epoch', type=int, default=280, help='load epoch model')
    parser.add_argument('--num_test_problem', type=int, default=100, help='#test instance')

    # ETC SETTINGS
    parser.add_argument('--input_sequencing_rule', type=str, default='random', help='use rule for input sequencing')
    parser.add_argument('--time_limit', type=int, default=600, help='time_limit')
    
    parser.add_argument('--rolling_foup_cnt', type=int, default=4, help='rolling horizon range')
    parser.add_argument('--show', type=bool, default=False, help='show schedule monitor')

    args = parser.parse_args([])
    args.z = []
    args.a = []
    
    # Purge setting
    base_score, drl_score, env = compare_performance(args)
    
    return base_score, drl_score, env

In [2]:
baseline, drl, env = main()

>>> trained Model ./saved_models/checkpoint_s4sp6.pt Loaded....
>>> use conventional robot sequence = swap


In [3]:
# print average results
print("="*50)
print("Dual-arm cluster tool with MLIF flow wafers")
print(f'Average makespan of Base: {baseline.mean():.2f}, DRL: {drl.mean():.2f}')
print("="*50)

Dual-arm cluster tool with MLIF flow wafers
Average makespan of Base: 11167.01, DRL: 10459.91


In [4]:
results = []

for instance_id in range(len(baseline)):
    type_1_process_time = env._process_time[instance_id,0,1:env.num_stage+1].tolist()
    type_2_process_time = env._process_time[instance_id,1,1:env.num_stage+1].tolist()
    type_3_process_time = env._process_time[instance_id,2,1:env.num_stage+1].tolist()
    type_4_process_time = env._process_time[instance_id,3,1:env.num_stage+1].tolist()
    type_5_process_time = env._process_time[instance_id,4,1:env.num_stage+1].tolist()
    
    results.append([
        instance_id,                    
        type_1_process_time, 
        type_2_process_time,
        type_3_process_time,
        type_4_process_time,
        type_5_process_time,
        int(baseline[instance_id]),          # baseline (SWAP) makespan
        int(drl[instance_id])           # DRL makespan
    ])

df = pd.DataFrame(results, columns=[
    "InstanceID", 
    "Type 1 Process Time",
    "Type 2 Process Time",
    "Type 3 Process Time",
    "Type 4 Process Time",
    "Type 5 Process Time",
    "SWAP Makespan", 
    "RL Makespan"
])

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

df


,InstanceID,Type 1 Process Time,Type 2 Process Time,Type 3 Process Time,Type 4 Process Time,Type 5 Process Time,SWAP Makespan,RL Makespan
0,0,"[43, 258, 87, 72]","[115, 136, 69, 127]","[62, 62, 98, 50]","[26, 81, 222, 76]","[141, 171, 275, 73]",10391,9317
1,1,"[22, 248, 72, 140]","[14, 41, 141, 13]","[84, 250, 54, 56]","[33, 225, 70, 124]","[129, 267, 274, 94]",12084,11067
2,2,"[110, 230, 280, 35]","[89, 114, 134, 128]","[56, 281, 68, 106]","[88, 141, 251, 39]","[140, 124, 261, 56]",12021,11274
3,3,"[73, 280, 112, 69]","[57, 168, 56, 124]","[104, 276, 214, 146]","[119, 98, 50, 35]","[73, 95, 131, 32]",11092,10012
4,4,"[67, 249, 38, 131]","[80, 114, 34, 108]","[132, 61, 163, 70]","[56, 212, 48, 55]","[139, 23, 241, 28]",10833,9873
5,5,"[140, 275, 62, 90]","[97, 17, 201, 128]","[128, 289, 202, 77]","[95, 21, 297, 49]","[59, 142, 105, 64]",11460,11011
6,6,"[30, 256, 239, 109]","[68, 166, 230, 63]","[89, 11, 60, 112]","[58, 266, 28, 46]","[96, 101, 225, 48]",10544,9869
7,7,"[110, 231, 160, 39]","[149, 198, 183, 25]","[44, 64, 142, 87]","[31, 212, 168, 53]","[83, 288, 122, 121]",11088,10320
8,8,"[109, 119, 277, 93]","[50, 275, 137, 21]","[61, 99, 243, 57]","[99, 204, 235, 63]","[89, 20, 130, 89]",11155,10752
9,9,"[84, 294, 257, 14]","[90, 118, 108, 29]","[105, 107, 232, 111]","[127, 98, 97, 89]","[11, 249, 118, 83]",10919,9865
